In [1]:
import pandas as pd

In [30]:
with open('../Data/coursera_sessions_train.txt', 'r') as f:
    train_data = f.readlines()

In [31]:
with open('../Data/coursera_sessions_test.txt', 'r') as f:
    test_data = f.readlines()

In [101]:
import numpy as np
def preprocess_data(data):
    tdata = map(lambda x: x.replace('\n',''), data)
    tdata = map(lambda x: x.split(';'), tdata)
    
    ttdata = []
    for el in tdata:
        if el[1]!='':
            ttdata.append(el)
            
    obs = np.array([x[0].split(',') for x in ttdata])
    buy = np.array([x[1].split(',') for x in ttdata])
    
    obs_all = []
    for el in obs:
        obs_all += el
        
    buy_all = []
    for el in buy:
        buy_all += el
    
    obs_un = np.unique(obs_all)
    buy_un = np.unique(buy_all)
    
    obs_counts = [obs_all.count(x) for x in obs_un]
    buy_counts = [buy_all.count(x) for x in buy_un]
    
    n_obs = [len(x) for x in obs]
    n_buy = [len(x) for x in buy]
    return {'obs': obs, 'obs_all': obs_all, 'obs_un': obs_un, 'obs_counts': obs_counts, 'n_obs': n_obs,
           'buy': buy, 'buy_all': buy_all, 'buy_un': buy_un, 'buy_counts': buy_counts, 'n_buy': n_buy}

In [102]:
train_sum = preprocess_data(train_data)

In [103]:
pop_obs = sorted(zip(train_sum['obs_un'], train_sum['obs_counts']), key=lambda x: x[1], reverse=True)
ppop_obs = [x[0] for x in pop_obs]
pop_buy = sorted(zip(train_sum['buy_un'], train_sum['buy_counts']), key=lambda x: x[1], reverse=True)
ppop_buy = [x[0] for x in pop_buy]

In [108]:
def prec_recall(obs, buy, ppop_obs, k):
    #print obs
    obs_un = np.unique(obs)
    #print obs_un
    inds = map(lambda x: ppop_obs.index(x), obs_un)
    indsr = map(lambda x: obs.index(x), obs_un)
    #print inds
    inds_obs = sorted(zip(inds, obs_un, indsr), key=lambda x: (x[0], x[2]))
    l = min(k, len(obs_un))
    recoms = [str(x[1]) for x in inds_obs][:l]
    #print recoms
    pr_sum = 0
    rec_sum = 0
    n_sum = 0
    for i in range(l):
        if recoms[i] in buy:
            n_sum += 1
            pr_sum += float(n_sum)/(i+1)
            rec_sum += float(n_sum)/len(buy)
    if n_sum ==0: n_sum = 1
    pr = float(pr_sum)/n_sum
    recall = float(rec_sum)/n_sum
    return (pr, recall)

In [109]:
prs = []
recalls = []
for i in range(len(train_sum['obs'])):
    pr, recall = prec_recall(train_sum['obs'][i], train_sum['buy'][i], ppop_obs, 1)
    prs.append(pr)
    recalls.append(recall)

In [110]:
print np.mean(prs)
print np.mean(recalls)

0.558203991131
0.481560050414


In [111]:
prs = []
recalls = []
for i in range(len(train_sum['obs'])):
    pr, recall = prec_recall(train_sum['obs'][i], train_sum['buy'][i], ppop_obs, 5)
    prs.append(pr)
    recalls.append(recall)
print np.mean(prs)
print np.mean(recalls)

0.700676736881
0.818842183469


In [98]:
def write_file(filename, res):
    with open(filename, 'w') as f:
        f.write(str(res))

In [100]:
write_file('recom_1.txt', ' '.join(map(str, [0.48, 0.56, 0.82, 0.7])))